In [11]:
import pandas as pd

In [12]:
cleaned_league = pd.read_csv("complete_cleaned.csv")
counter_df = pd.read_csv("champs_counters_wr.csv")
cleaned_league.head()

,Unnamed: 0,year,gameid,league,teamname,side,ban1,ban2,ban3,ban4,ban5,pick1,pick2,pick3,pick4,pick5,result
0,0,2017,1506-1540,LPL,I May,Blue,Syndra,Malzahar,Ashe,Karma,Poppy,Maokai,Kha'Zix,Cassiopeia,Varus,Tahm Kench,1
1,1,2017,1506-1540,LPL,Royal Never Give Up,Red,Camille,Rengar,Zyra,Elise,Rek'Sai,Kled,Lee Sin,Ryze,Caitlyn,Nautilus,0
2,2,2017,1506-1541,LPL,I May,Blue,Syndra,Malzahar,Ashe,Rek'Sai,Kha'Zix,Maokai,Lee Sin,Corki,Caitlyn,Thresh,1
3,3,2017,1506-1541,LPL,Royal Never Give Up,Red,Rengar,Camille,Varus,Cassiopeia,Orianna,Trundle,Rumble,Ryze,Jhin,Zyra,0
4,4,2017,1507-1544,LPL,Invictus Gaming,Blue,Jayce,Elise,Malzahar,Kha'Zix,Lee Sin,Singed,Rengar,LeBlanc,Varus,Tahm Kench,1


In [13]:
cleaned_league.isna().sum()

Unnamed: 0       0
year             0
gameid           0
league           0
teamname        27
side             0
ban1          2053
ban2          1830
ban3          2011
ban4          2021
ban5          2307
pick1            0
pick2            0
pick3            0
pick4            0
pick5            0
result           0
dtype: int64

In [14]:
cleaned_league = cleaned_league.drop(columns=["Unnamed: 0"])

In [16]:
import pandas as pd
import ast  # For safely parsing tuple strings

def format_champion(champ):
    if pd.isna(champ):  
        return None  
    return champ.lower().replace(" ", "-").replace("'", "").replace("nunu-&-willump", "nunu-willump").replace("dr.-mundo", "dr-mundo")

def get_picks_and_bans(df): 
    blue_picks = df[df['side'] == 'Blue'][['pick1', 'pick2', 'pick3', 'pick4', 'pick5']].values.flatten().tolist()
    red_picks = df[df['side'] == 'Red'][['pick1', 'pick2', 'pick3', 'pick4', 'pick5']].values.flatten().tolist()

    blue_bans = df[df['side'] == 'Blue'][['ban1', 'ban2', 'ban3', 'ban4', 'ban5']].values.flatten().tolist()
    red_bans = df[df['side'] == 'Red'][['ban1', 'ban2', 'ban3', 'ban4', 'ban5']].values.flatten().tolist()

    blue_picks = [format_champion(champ) for champ in blue_picks if pd.notna(champ)]
    red_picks = [format_champion(champ) for champ in red_picks if pd.notna(champ)]
    
    blue_bans = [format_champion(champ) for champ in blue_bans if pd.notna(champ)]
    red_bans = [format_champion(champ) for champ in red_bans if pd.notna(champ)]

    return blue_picks, red_picks, blue_bans, red_bans

def preprocess_counters(counter_df):
    counter_dict = {}
    wr_dict = {}  # Store win rates for each champion

    for _, row in counter_df.iterrows():
        champ = format_champion(row['Champion'])
        wr_dict[champ] = row['champ_wr']  # Store win rate
        
        if pd.notna(row['counters champ']) and isinstance(row['counters champ'], str):
            try:
                counters = ast.literal_eval(row['counters champ'])  # Safely parse the tuple
                if isinstance(counters, tuple):  # Ensure it's a tuple before extracting
                    if champ not in counter_dict:
                        counter_dict[champ] = set()
                    counter_dict[champ].add(format_champion(counters[0]))  # Extract champion names
            except (SyntaxError, ValueError):
                continue  # Skip malformed rows
                
    return counter_dict, wr_dict  # Return both counter and win rate dictionaries

def count_counters(df, counter_dict): 
    blue_picks, red_picks, blue_bans, red_bans = get_picks_and_bans(df)

    num_blue_counters = sum(1 for red_champ in red_picks for blue_champ in blue_picks if blue_champ in counter_dict.get(red_champ, set()))
    num_red_counters = sum(1 for blue_champ in blue_picks for red_champ in red_picks if red_champ in counter_dict.get(blue_champ, set()))
    
    num_blue_counters_banned = sum(1 for blue_champ in blue_picks for ban in blue_bans if ban in counter_dict.get(blue_champ, set()))
    num_red_counters_banned = sum(1 for red_champ in red_picks for ban in red_bans if ban in counter_dict.get(red_champ, set()))

    return pd.Series({
        "Red Counters": num_red_counters, 
        "Blue Counters": num_blue_counters,
        "Red Counters Banned": num_red_counters_banned,
        "Blue Counters Banned": num_blue_counters_banned
    })

def calculate_mean_wr(df, wr_dict):
    blue_picks, red_picks, _, _ = get_picks_and_bans(df)

    # Get WR values for each pick, default to 0.5 if missing
    blue_wr = [wr_dict.get(champ, 0.5) for champ in blue_picks]
    red_wr = [wr_dict.get(champ, 0.5) for champ in red_picks]

    mean_blue_wr = sum(blue_wr) / len(blue_wr) if blue_wr else 0
    mean_red_wr = sum(red_wr) / len(red_wr) if red_wr else 0

    return pd.Series({"Mean Blue WR": mean_blue_wr, "Mean Red WR": mean_red_wr})

def process_data(df, counter_df):
    counter_dict, wr_dict = preprocess_counters(counter_df)
    
    # Compute counters and merge
    counters_df = df.groupby("gameid").apply(lambda x: count_counters(x, counter_dict)).reset_index()
    df = df.merge(counters_df, on="gameid", how="left")
    
    df["num_counters_picked"] = df.apply(lambda row: row["Red Counters"] if row["side"] == "Red" else row["Blue Counters"], axis=1)
    df["num_counters_banned"] = df.apply(lambda row: row["Red Counters Banned"] if row["side"] == "Red" else row["Blue Counters Banned"], axis=1)
    
    df["num_counters_picked"] = df["num_counters_picked"].fillna(0)
    df["num_counters_banned"] = df["num_counters_banned"].fillna(0)
    
    # Calculate PGA
    df["PGA"] = df["num_counters_picked"] + df["num_counters_banned"]
    
    df.drop(columns=["Red Counters", "Blue Counters", "Red Counters Banned", "Blue Counters Banned"], inplace=True)

    # Compute mean WR for each game and merge
    wr_df = df.groupby("gameid").apply(lambda x: calculate_mean_wr(x, wr_dict)).reset_index()
    df = df.merge(wr_df, on="gameid", how="left")

    # Assign WR values based on team side
    df["mean_champ_wr"] = df.apply(lambda row: row["Mean Red WR"] if row["side"] == "Red" else row["Mean Blue WR"], axis=1)

    df.drop(columns=["Mean Red WR", "Mean Blue WR"], inplace=True)

    # Compute which team had the higher PGA
    df['higher_PGA'] = df.groupby('gameid')['PGA'].transform(lambda x: (x == x.max()).astype(int))

    # Remove games where both teams have the same PGA
    same_pga_games = df.groupby('gameid')['PGA'].nunique() == 1
    df = df[~df['gameid'].isin(same_pga_games[same_pga_games].index)]
    
    return df  

final = process_data(cleaned_league, counter_df)
final.to_csv('transformed_data.csv')
final

C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_4132\3753962636.py:76: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  counters_df = df.groupby("gameid").apply(lambda x: count_counters(x, counter_dict)).reset_index()
C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_4132\3753962636.py:91: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  wr_df = df.groupby("gameid").apply(lambda x: calculate_mean_wr(x, wr_dict)).rese

,year,gameid,league,teamname,side,ban1,ban2,ban3,ban4,ban5,...,pick2,pick3,pick4,pick5,result,num_counters_picked,num_counters_banned,PGA,mean_champ_wr,higher_PGA
0,2017,1506-1540,LPL,I May,Blue,Syndra,Malzahar,Ashe,Karma,Poppy,...,Kha'Zix,Cassiopeia,Varus,Tahm Kench,1,0,2,2,0.508666,1
1,2017,1506-1540,LPL,Royal Never Give Up,Red,Camille,Rengar,Zyra,Elise,Rek'Sai,...,Lee Sin,Ryze,Caitlyn,Nautilus,0,0,0,0,0.493086,0
2,2017,1506-1541,LPL,I May,Blue,Syndra,Malzahar,Ashe,Rek'Sai,Kha'Zix,...,Lee Sin,Corki,Caitlyn,Thresh,1,0,0,0,0.510833,0
3,2017,1506-1541,LPL,Royal Never Give Up,Red,Rengar,Camille,Varus,Cassiopeia,Orianna,...,Rumble,Ryze,Jhin,Zyra,0,1,0,1,0.492558,1
4,2017,1507-1544,LPL,Invictus Gaming,Blue,Jayce,Elise,Malzahar,Kha'Zix,Lee Sin,...,Rengar,LeBlanc,Varus,Tahm Kench,1,0,1,1,0.492380,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119955,2025,LOLTMNT06_96854,LFL,Vitality.Bee,Red,Skarner,Viktor,Maokai,Rell,Lux,...,Ivern,Sylas,Varus,Nautilus,1,1,1,2,0.497897,1
119956,2025,LOLTMNT06_96867,LFL2,Zerance,Blue,Aurora,Karthus,Kayn,Galio,Irelia,...,Xin Zhao,Sylas,Corki,Nautilus,0,1,2,3,0.501714,1
119957,2025,LOLTMNT06_96867,LFL2,Project Conquerors,Red,Viktor,K'Sante,Ivern,Jayce,Akali,...,Amumu,Ambessa,Varus,Rell,1,2,0,2,0.509128,0
119960,2025,LOLTMNT06_96906,LFL2,IZI Dream,Blue,Akali,Rell,Varus,Skarner,Smolder,...,Maokai,Corki,Kai'Sa,Rakan,1,1,2,3,0.516646,1


In [10]:
final.isna().sum()

year                      0
gameid                    0
teamname                 22
side                      0
ban1                   1376
ban2                   1206
ban3                   1356
ban4                   1351
ban5                   1562
pick1                     0
pick2                     0
pick3                     0
pick4                     0
pick5                     0
result                    0
num_counters_picked       0
num_counters_banned       0
PGA                       0
mean_champ_wr             0
higher_PGA                0
dtype: int64